<a href="https://colab.research.google.com/github/RomersonMonteiro/Arquivos-DNC/blob/main/Mini_Projeto_Wallmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center><font color='green'><b>Deliverable 02 - Slot 04</b></font></center>

<center>Mini Projeto - Wallmart</center>

## <font color='#FF4500'>1 - **Importando:**</font>

### 1.1 - Bibliotecas utilizadas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     
import seaborn as sns

import zipfile as zip
import requests
import io

import scipy.stats as stats
from shapely.geometry import Point, Polygon
from datetime import datetime

## <font color='#FF4500'>2 - **A Coleta de Dados:**</font>

### 2.1 - Dados do Kaggle

In [ ]:
#FONTE: 'https://www.kaggle.com/yasserh/walmart-dataset'
url = "https://raw.githubusercontent.com/leokoki/Data-Expert-DNC/main/Day%204/Data/Walmart.csv"
df = pd.read_csv(url)
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


### 2.2 - Dados dos Feriados

#### 2.2.1 - Verifica Range de Data no DataSet

In [ ]:
df.Date.min(),df.Date.max()

('01-04-2011', '31-12-2010')

#### 2.2.2 -  Cria o DataSet de Holyday e importa ele do GitLab

In [ ]:
url_he = "https://raw.githubusercontent.com/leokoki/Data-Expert-DNC/main/Day%204/Data/HolidayEvents.csv"
he = pd.read_csv(url_he,sep=';')
he.head()

,Date,Holiday
0,12/02/2010,Super Bowl
1,10/09/2010,Labour Day
2,26/11/2010,Thanksgiving
3,31/12/2010,Christmas
4,12/02/2011,Super Bowl


#### 2.2.3 - Validando as Datas p/ Realimentar a base caso necessário

In [ ]:
set(df[~df.Date.isin(he.Date) & df.Holiday_Flag == 1].Date)

{'07-09-2012',
 '09-09-2011',
 '10-02-2012',
 '10-09-2010',
 '11-02-2011',
 '12-02-2010',
 '25-11-2011',
 '26-11-2010',
 '30-12-2011',
 '31-12-2010'}

## <font color='#FF4500'>3 - **Tratamento de Dados:**</font>

### 3.1 - Analise do que será tratado

#### 3.1.1 - Verificação de Data Types e se há ocorrência de Nulos

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [ ]:
he.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     12 non-null     object
 1   Holiday  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


#### 3.1.2 - Validação de Duplicatas

In [ ]:
df.duplicated().value_counts()

False    6435
dtype: int64

### 3.2 - Tratamento de Dados

#### 3.2.1 - Ajustando variáveis

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         6435 non-null   int64         
 1   Date          6435 non-null   datetime64[ns]
 2   Weekly_Sales  6435 non-null   float64       
 3   Holiday_Flag  6435 non-null   int64         
 4   Temperature   6435 non-null   float64       
 5   Fuel_Price    6435 non-null   float64       
 6   CPI           6435 non-null   float64       
 7   Unemployment  6435 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 402.3 KB


In [ ]:
he['Date'] = pd.to_datetime(he['Date'],format='%d/%m/%Y')
he.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     12 non-null     datetime64[ns]
 1   Holiday  12 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 320.0+ bytes


#### 3.2.2 -  Separando Ano | Mês | Dia | Dia da Semana

In [ ]:
# Year
df['Year'] = df['Date'].dt.strftime("%Y")
# Month
df['Month'] = df['Date'].dt.strftime("%m")
# Day
df['Day'] = df['Date'].dt.strftime("%d")
# Week Day
df['WeekDay'] = df['Date'].dt.strftime("%A")
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,WeekDay
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,2010,05,02,Sunday
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106,2010,12,02,Thursday
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,02,19,Friday
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,02,26,Friday
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106,2010,05,03,Monday


### 3.3 - Junção do DataSet (Feriados + Wallmart)

In [ ]:
he.head()

,Date,Holiday
0,2010-02-12,Super Bowl
1,2010-09-10,Labour Day
2,2010-11-26,Thanksgiving
3,2010-12-31,Christmas
4,2011-02-12,Super Bowl


In [ ]:
df = df.join(he.set_index('Date'),on='Date',how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         6435 non-null   int64         
 1   Date          6435 non-null   datetime64[ns]
 2   Weekly_Sales  6435 non-null   float64       
 3   Holiday_Flag  6435 non-null   int64         
 4   Temperature   6435 non-null   float64       
 5   Fuel_Price    6435 non-null   float64       
 6   CPI           6435 non-null   float64       
 7   Unemployment  6435 non-null   float64       
 8   Year          6435 non-null   object        
 9   Month         6435 non-null   object        
 10  Day           6435 non-null   object        
 11  WeekDay       6435 non-null   object        
 12  Holiday       135 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(5)
memory usage: 653.7+ KB


### 3.4 - Update dos feriados que não foram disponibilizados no DataSet

#### 3.4.1 - Lista de Datas que estão fora do DataSet de Feriados, porém contam do DataSet da Wallmart como Feriado

In [ ]:
list(set(df[~df.Date.isin(he.Date) & df.Holiday_Flag == 1].Date))

[Timestamp('2011-11-25 00:00:00'),
 Timestamp('2012-10-02 00:00:00'),
 Timestamp('2011-12-30 00:00:00'),
 Timestamp('2010-10-09 00:00:00'),
 Timestamp('2011-11-02 00:00:00'),
 Timestamp('2011-09-09 00:00:00'),
 Timestamp('2012-07-09 00:00:00'),
 Timestamp('2010-12-02 00:00:00')]

#### 3.4.2 - Atualizando a Flag para 0

In [ ]:
df.loc[df.Date.isin(list(set(df[~df.Date.isin(he.Date) & df.Holiday_Flag == 1].Date))),'Holiday_Flag']=0

#### 3.4.3 - Validação se foi atualizada a Flag ou Não

In [ ]:
df[df.Date.isin(list(set(df[~df.Date.isin(he.Date) & df.Holiday_Flag == 1].Date)))]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,WeekDay,Holiday


In [ ]:
list(set(df[~df.Date.isin(he.Date) & df.Holiday_Flag == 1].Date))

[]

## <font color='#FF4500'>**4 - Metodologia**</font>

### 4.1 - Qual o preço médio de venda semanal por loja?

In [ ]:
df.groupby("Store").agg("Weekly_Sales":"mean")


SyntaxError: ignored

### 4.2 - Qual o atributo que tem maior correlação (positiva ou negativa) com o número de vendas? (E que faça sentido)

### 4.3 - Da loja com maior venda acumulada (soma de vendas de todo o período), quantas semanas do ano ele ultrapassou a média do período?

### 4.4 - Faça um gráfico com as vendas máximas, mínimas e médias de todas as lojas ao longo do tempo. Ressalte de alguma forma, o período de feriados mencionados acima.

## <font color='#FF4500'>5 - **Análise de Insights**</font>

## <font color='#FF4500'>6 - **Resultado:**</font>